<a href="https://colab.research.google.com/github/yichong96/AI_Clinician/blob/master/BERT_Extractive_Summ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

     |████████████████████████████████| 2.1MB 5.8MB/s 
     |████████████████████████████████| 3.2MB 20.0MB/s 
     |████████████████████████████████| 890kB 41.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=23c43dc66aa7cc1fed1be6f9de5dbf120ac8a1d38fad94a62779a88180d4c531
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [17]:
with open('/content/example1.txt', 'r') as f:
  text = f.read().strip() # contains the whole TED talk without line spaces

In [22]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [23]:
sentences = sent_tokenize(text)

In [24]:
sentences

["I've spent the last couple of years traveling around the world giving talks to big corporations and little bitty start-ups and lots of leadership teams and women's groups, and what I've been talking to people about, I've been trying really hard to convince people that we can change the way we work.But every time I do a talk, somebody comes backstage or follows me offstage and says, You know, I'm so inspired by what you say.",
 "It's so great, it makes so much sense.",
 "But we can't.",
 "We can't because we're regulated.",
 "We can't because our CFO says we can't do it.",
 "We can't because we're in Europe.",
 "“We can’t because we’re a service industry.” We can't because we're a nonprofit.",
 'And then last year came the pandemic.',
 'And the pandemic changed everything all over the world.Service people started realizing that they had to suit up and wear masks and take temperatures and wash their hands.',
 'We had to start standing six feet apart in lines.',
 'We started working fro

In [25]:
# load BERT model
from transformers import BertModel, BertTokenizer
model = BertModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [31]:
sentence_encodings = tokenizer(sentences, truncation=True, padding=True, return_tensors='pt')
print(sentence_encodings['input_ids'][0])

tensor([  101,  1045,  1005,  2310,  2985,  1996,  2197,  3232,  1997,  2086,
         7118,  2105,  1996,  2088,  3228,  7566,  2000,  2502, 11578,  1998,
         2210,  2978,  3723,  2707,  1011, 11139,  1998,  7167,  1997,  4105,
         2780,  1998,  2308,  1005,  1055,  2967,  1010,  1998,  2054,  1045,
         1005,  2310,  2042,  3331,  2000,  2111,  2055,  1010,  1045,  1005,
         2310,  2042,  2667,  2428,  2524,  2000,  8054,  2111,  2008,  2057,
         2064,  2689,  1996,  2126,  2057,  2147,  1012,  2021,  2296,  2051,
         1045,  2079,  1037,  2831,  1010,  8307,  3310, 20212,  2030,  4076,
         2033, 12446, 26702,  1998,  2758,  1010,  2017,  2113,  1010,  1045,
         1005,  1049,  2061,  4427,  2011,  2054,  2017,  2360,  1012,   102])


In [33]:
outputs = model(**sentence_encodings, output_hidden_states = True)

In [49]:
print(outputs.hidden_states[0].shape)
print(outputs.pooler_output.shape)
print(len(outputs.hidden_states))
print(outputs.hidden_states[0].shape)

torch.Size([77, 100, 768])
torch.Size([77, 768])
13
torch.Size([77, 100, 768])


In [48]:
# Use pooler output shape for clustering (for now)

13


In [69]:
import numpy as np
from sklearn.cluster import KMeans
k = 20
sentence_embeddings = outputs.pooler_output
X = np.array(sentence_embeddings.detach().numpy())
kmeans = KMeans(n_clusters=k, random_state=0).fit(X)

In [70]:
from sklearn.neighbors import NearestNeighbors
labels = kmeans.labels_
cluster_centers = kmeans.cluster_centers_

neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(sentence_embeddings.detach().numpy(), labels)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                 radius=1.0)

In [71]:
sentence_indices = []
neigh, index = neigh.kneighbors(cluster_centers)

In [72]:
index = index.flatten()

In [73]:
index.sort()

In [74]:
summary = []
for i in index:
  summary.append(sentences[i])

In [75]:
print(' '.join(summary))

We had to start standing six feet apart in lines. We started working virtually. And we started learning all kinds of things because we had to. The family is the toddler walking around behind you in the Zoom call with the pet. And that when we used to say, Well, this is my work home and this is my family home, and those are two completely different things, for many of us, it's exactly the same thing. For many of us, work is at home and the home is -- and it's confusing, and it's creating a whole different level of complexity and coordination so that we understand that it's easier actually to work when we can separate the work that we do as a team from the work that we do in our family.Furthermore, in order to be able to do all that, we have to recognize that we're all adults. We don't care what Google's doing because we're not Google. We don't care what some other company is doing. We're all figuring it out as we go along and we're figuring it out for our organizations in our teams at t